In [1]:
import pandas as pd
import numpy as np

In [4]:
world_pop_df = pd.read_csv("../data/worldcitiespop.csv")

/var/folders/1x/k0shl5v96p77c63jw7cl3xqm0000gn/T/ipykernel_29679/1030460779.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  world_pop_df = pd.read_csv("../data/worldcitiespop.csv")


In [9]:
world_pop_df["Country"].unique()

array(['ad', 'ae', 'af', 'ag', 'ai', 'al', 'am', 'an', 'ao', 'ar', 'at',
       'au', 'aw', 'az', 'ba', 'bb', 'bd', 'be', 'bf', 'bg', 'bh', 'bi',
       'bj', 'bm', 'bn', 'bo', 'br', 'bs', 'bt', 'bw', 'by', 'bz', 'ca',
       'cc', 'cd', 'cf', 'cg', 'ch', 'ci', 'ck', 'cl', 'cm', 'cn', 'co',
       'cr', 'cu', 'cv', 'cx', 'cy', 'cz', 'de', 'dj', 'dk', 'dm', 'do',
       'dz', 'ec', 'ee', 'eg', 'eh', 'er', 'es', 'et', 'fi', 'fj', 'fk',
       'fm', 'fo', 'fr', 'ga', 'gb', 'gd', 'ge', 'gf', 'gg', 'gh', 'gi',
       'gl', 'gm', 'gn', 'gp', 'gq', 'gr', 'gs', 'gt', 'gw', 'gy', 'hk',
       'hn', 'hr', 'ht', 'hu', 'id', 'ie', 'il', 'im', 'in', 'iq', 'ir',
       'is', 'it', 'je', 'jm', 'jo', 'jp', 'ke', 'kg', 'kh', 'ki', 'km',
       'kn', 'kp', 'kr', 'kw', 'ky', 'kz', 'la', 'lb', 'lc', 'li', 'lk',
       'lr', 'ls', 'lt', 'lu', 'lv', 'ly', 'ma', 'mc', 'md', 'me', 'mg',
       'mh', 'mk', 'ml', 'mm', 'mn', 'mo', 'mp', 'mq', 'mr', 'ms', 'mt',
       'mu', 'mv', 'mw', 'mx', 'my', 'mz', 'na', 'n

In [20]:
us_cities_df = world_pop_df[world_pop_df["Country"] == "us"]

In [21]:
us_cities_df[:3]

,Country,City,AccentCity,Region,Population,Latitude,Longitude
2907718,us,abanda,Abanda,AL,NaN,33.100833,-85.529722
2907719,us,abbeville,Abbeville,AL,NaN,31.571667,-85.250556
2907720,us,abbot springs,Abbot Springs,AL,NaN,33.360833,-86.481667


In [22]:
# Count Number of Cities with NaN Population v/s Non-NaN Population
us_cities_df["Population"].isnull().sum(), us_cities_df["Population"].notnull().sum()

(np.int64(137814), np.int64(4175))

In [2]:
us_cities_df = pd.read_csv("../data/us_cities.csv")

In [5]:
df_to_write = us_cities_df.drop(
    columns=["Country", "Latitude", "Longitude"], inplace=False
)
df_to_write[:3]

,City,AccentCity,Region,Population
0,abanda,Abanda,AL,NaN
1,abbeville,Abbeville,AL,NaN
2,abbot springs,Abbot Springs,AL,NaN


In [6]:
import redis

In [19]:
redis_client: redis.StrictRedis = redis.Redis(host="localhost", port=6379, db=0)

In [ ]:
# Bulk insert function
from time import perf_counter


def insert_into_redis(df, redis_client):
    start = perf_counter()
    pipeline = redis_client.pipeline()
    # Compute the pipeline for each row in the DataFrame
    for _, row in df.iterrows():
        # Define the unique key for the city with state included
        city_key = f"city:{row['City']}:{row['Region']}"

        pipeline.hset(city_key, "state", row["Region"])
        pipeline.hset(city_key, "population", row["Population"])

        # Add the city name to the state's Set for quick state-based lookups
        state_key = f"state:{row['Region']}"
        pipeline.sadd(state_key, row["City"])

    pipeline.execute()
    end = perf_counter()
    print(f"Time taken to insert {len(df)} records: {end-start} seconds")

In [ ]:
insert_into_redis(df_to_write, redis_client)

Time taken to insert 141989 records: 5.413582707988098 seconds


In [36]:
df_to_write["Region"].unique()

array(['AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL',
       'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME',
       'MH', 'MD', 'MA', 'MI', 'FM', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV',
       'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'MP', 'OH', 'OK', 'OR', 'PW',
       'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VI', 'VA',
       'WA', 'WV', 'WI', 'WY'], dtype=object)

In [39]:
df_to_write[df_to_write["Region"] == "CA"].dropna(how="any")[:10]

,City,AccentCity,Region,Population
10160,adelanto,Adelanto,CA,21955.0
10170,agoura hills,Agoura Hills,CA,22222.0
10183,alameda,Alameda,CA,70443.0
10185,alamo,Alamo,CA,17764.0
10189,albany,Albany,CA,16144.0
10203,alhambra,Alhambra,CA,88386.0
10208,aliso viejo,Aliso Viejo,CA,41314.0
10220,alondra park,Alondra Park,CA,8028.0
10223,alpine,Alpine,CA,15391.0
10239,altadena,Altadena,CA,43226.0


In [40]:
redis_client.sismember("state:CA", "los angeles")

True

In [49]:
redis_client.srandmember("state:CA", number=5)

[b'north star ranch', b'apex', b'gateley', b'robinsons corner', b'austin']

In [24]:
df_to_write[:3]

,City,AccentCity,Region,Population
0,abanda,Abanda,AL,NaN
1,abbeville,Abbeville,AL,NaN
2,abbot springs,Abbot Springs,AL,NaN


In [ ]:
unique_states = df_to_write["Region"].unique()

# Save unique states into Redis HSET
pipeline = redis_client.pipeline()
for state in unique_states:
    pipeline.sadd("states", state)
pipeline.execute()